In [ ]:
#Import the dependencies 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
#from scipy import stats
from windrose import WindroseAxes
import os

In [ ]:
# Load dataset 
#df = pd.read_csv('data/seraleon_raw.csv')
df=pd.read_csv('F:/solar-challenge-week1/data/seraleon_raw.csv')

In [ ]:
# Convert Timestamp to datetime
df['Timestamp'] = pd.to_datetime(df['Timestamp'])

In [ ]:
# 1. Summary Statistics & Missing-Value Report
print("The description of the numeric columns:")
print(df.describe())
#Overview of categorical columns
#print("***************************")
#print("The description of the categorical columns: ")
#print(df.describe(include=['object']))
#Overview of all columns
print("****************************")
print("The description of all columns:")
print (df.describe(include='all'))
#Overview of the dataset
print("****************************")
print("An overview of the dataset: the first 5 rows") 
print(df.head())
#Overview of the dataset
print("****************************")
print("An overview of the dataset: the last 5 rows")

# Overview of numeric columns
print("The description of the numeric columns:") 
print (df.describe())
#Overview of categorical columns
print("****************************")
print("The description of the categorical columns:") 
#print(df.describe(include=['object']))
#Overview of all columns
print("****************************")
print("The description of all columns:")
print (df.describe(include='all'))
#Overview of the dataset
print("****************************")
print("An overview of the dataset: the first 5 rows") 
print(df.head())
#Overview of the dataset
print("****************************")
print("An overview of the dataset: the last 5 rows")
print(df.tail())
#Overview of the dataset
print("****************************")
print("An overview of the dataset: a random sample of 5 rows")
print(df.sample())
#Overview of the dataset
print("****************************")
print("An overview of the dataset: the shape of the dataset")
print(df.info())
#Check for missing values
print("****************************")
print("Check for missing values: ")
print(df.isna().sum())


In [ ]:
# 2. Outlier Detection & Basic Cleaning
key_columns = ['GHI', 'DNI', 'DHI', 'ModA', 'ModB', 'WS', 'WSgust']
print("\n=== Outlier Detection (Z-scores > 3) ===")
for col in key_columns:
    df[f'{col}_zscore'] = np.abs(stats.zscore(df[col], nan_policy='omit'))
outliers = df[df[[f'{col}_zscore' for col in key_columns]].gt(3).any(axis=1)]
print(f"Number of rows with outliers (|Z|>3): {len(outliers)}")

In [ ]:
## Impute missing values in key columns with median
for col in key_columns:
    df[col].fillna(df[col].median(), inplace=True)

# Drop rows with missing Timestamp or other critical non-numeric columns if any
df.dropna(subset=['Timestamp'], inplace=True)


In [ ]:
# Export cleaned DataFrame
if not os.path.exists('data'):
    os.makedirs('data')
df.drop(columns=[f'{col}_zscore' for col in key_columns]).to_csv('output/Seraleon_clean.csv', index=False)
print("\nCleaned data exported to data/benin_clean.csv")


In [ ]:
 # Histograms
plt.figure(figsize=(16, 4))
plt.subplot(1, 3, 1)
sns.histplot(df['GHI'], bins=30, kde=True)
plt.title('Distribution of GHI')
plt.xlabel('GHI')
plt.subplot(1, 3, 2)
sns.histplot(df['WSgust'], bins=30, kde=True)
plt.title('Distribution of WSgust')
plt.xlabel('WSgust')
plt.subplot(1, 3, 3)
sns.histplot(df['WS'], bins=30, kde=True)
plt.title('Distribution of Wind Speed')
plt.xlabel('WS')
plt.tight_layout()
plt.savefig('figures/histograms.png')
plt.show()

In [ ]:
# 3. Time Series Analysis
plt.figure(figsize=(12, 6))
for col in ['GHI', 'DNI', 'DHI', 'Tamb']:
    plt.plot(df['Timestamp'], df[col], label=col)
plt.title('Time Series of Solar Irradiance and Temperature')
plt.xlabel('Timestamp')
plt.ylabel('Value')
plt.legend()
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig('figure/time_series.png')
plt.show()

In [ ]:
# Monthly trends
df['Month'] = df['Timestamp'].dt.month
monthly_avg = df.groupby('Month')[['GHI', 'DNI', 'DHI', 'Tamb']].mean()
monthly_avg.plot(kind='bar', figsize=(12, 6))
plt.title('Monthly Average Solar Irradiance and Temperature')
plt.xlabel('Month')
plt.ylabel('Average Value')
plt.tight_layout()
plt.savefig('figures/monthly_trends.png')
plt.show()

In [ ]:
# 4. Cleaning Impact
cleaning_impact = df.groupby('Cleaning')[['ModA', 'ModB']].mean()
cleaning_impact.plot(kind='bar', figsize=(10, 5))
plt.title('Average ModA and ModB Pre/Post Cleaning')
plt.xlabel('Cleaning (0 = No, 1 = Yes)')
plt.ylabel('Average Value')
plt.tight_layout()
plt.savefig('figures/cleaning_impact.png')
plt.show()

In [ ]:
# 5. Correlation & Relationship Analysis
plt.figure(figsize=(10, 8))
corr = df[['GHI', 'DNI', 'DHI', 'TModA', 'TModB']].corr()
sns.heatmap(corr, annot=True, cmap='coolwarm', vmin=-1, vmax=1)
plt.title('Correlation Heatmap')
plt.tight_layout()
plt.savefig('figures/correlation_heatmap.png')
plt.show()

In [ ]:
# Scatter plots
plt.figure(figsize=(12, 4))
plt.subplot(1, 3, 1)
plt.scatter(df['WS'], df['GHI'], alpha=0.1)
plt.xlabel('Wind Speed (WS)')
plt.ylabel('GHI')
plt.subplot(1, 3, 2)
plt.scatter(df['WSgust'], df['GHI'], alpha=0.1)
plt.xlabel('Wind Gust (WSgust)')
plt.ylabel('GHI')
plt.subplot(1, 3, 3)
plt.scatter(df['WD'], df['GHI'], alpha=0.1)
plt.xlabel('Wind Direction (WD)')
plt.ylabel('GHI')
plt.tight_layout()
plt.savefig('figures/wind_vs_ghi.png')
plt.show()

plt.figure(figsize=(8, 4))
plt.subplot(1, 2, 1)
plt.scatter(df['RH'], df['Tamb'], alpha=0.5)
plt.xlabel('Relative Humidity (RH)')
plt.ylabel('Ambient Temperature (Tamb)')
plt.subplot(1, 2, 2)
plt.scatter(df['RH'], df['GHI'], alpha=0.5)
plt.xlabel('Relative Humidity (RH)')
plt.ylabel('GHI')
plt.tight_layout()
plt.savefig('figures/rh_vs_tamb_ghi.png')
plt.show()

In [ ]:
# 6. Wind & Distribution Analysis
# Wind rose plot
fig = plt.figure(figsize=(8, 8))
ax = WindroseAxes.from_ax(fig=fig)
ax.bar(df['WD'], df['WS'], normed=True, opening=0.8, edgecolor='white')
ax.set_legend()
plt.title('Wind Rose Plot (Wind Speed and Direction)')
plt.savefig('figures/wind_rose.png')
plt.show()

In [ ]:
## Temperature Analysis
print("\n=== Temperature and RH Analysis ===")
print("Correlation between RH and Tamb:", df['RH'].corr(df['Tamb']))
print("Correlation between RH and GHI:", df['RH'].corr(df['GHI']))

In [ ]:
# 8. Bubble Chart
plt.figure(figsize=(10, 6))
plt.scatter(df['Tamb'], df['GHI'], s=df['RH']*3, alpha=0.5)
plt.xlabel('Ambient Temperature (Tamb)')
plt.ylabel('GHI')
plt.title('GHI vs. Tamb (Bubble Size = RH)')
plt.tight_layout()
plt.savefig('figures/bubble_chart.png')
plt.show()